In [ ]:
import os 
import json
os.environ["HF_HOME"] = "/your/path/hf_cache"
from datasets import load_dataset 

    

In [ ]:
# Using QA dialogues, get the answer for specific questions
samples = []

keywords = ['how many people', 'How many people', 'how many persons', 'How many persons', 'how many person', 'How many person']#, \
    #'how many folks', 'How many folks', \
    #'how many individuals', 'How many individuals', 'how many individual', 'How many individual', \
    #'count of people', 'Count of people', 'number of people', 'Number of people', \
    #'count of person', 'Count of person', 'number of person', 'Number of person', \
    #]

In [ ]:
samples = []
with open("v7w_tell.json", "r") as f:
    data = json.load(f)
for exp in data:
    qa = exp["qa_pairs"]
    for i in qa:
        qu = i["question"]
        if any(k in qu for k in keywords) and exp["split"] == "val":
            sample = {}
            sample["image_id"]= exp["image_id"]
            sample["question"] = qu
            sample["answer"] = i["answer"]
            samples.append(sample)

In [ ]:
samples = sorted(samples, key=lambda x: x["image_id"])
os.makedirs("people_num", exist_ok=True)
# Save the samples to a JSON file
with open("people_num/dialogs_v7w.json", "w") as f:
    json.dump(samples, f, indent=4)

In [ ]:

# Loading the filtered annotations
data = json.load(open("people_num/dialogs_v7w.json"))
# Below is just a list of image ids that are not good samples
not_good_samples = json.load(open("people_num/not_good_samples.json"))
for i in data:
    if i["image_id"] in not_good_samples:
        data.remove(i)

In [ ]:
# Directly using the filtered annotations(We can manually check the filtered images while deleting the improper annotations)

for i, q in enumerate(data):
    if q["answer"] == "1." or q["answer"] == "one" or q["answer"] == "One" or q["answer"] == "one." or q["answer"] == "One." or q["answer"] == "one person" or q["answer"] == "One person" or q["answer"] == "one person." or q["answer"] == "One person.":
        data[i]["answer"] = "1"
    elif q["answer"] == "2." or q["answer"] == "two" or q["answer"] == "Two" or q["answer"] == "two." or q["answer"] == "Two." or q["answer"] == "two people" or q["answer"] == "Two people" or q["answer"] == "two people." or q["answer"] == "Two people.":
        data[i]["answer"] = "2"
    elif q["answer"] == "3." or q["answer"] == "three" or q["answer"] == "Three" or q["answer"] == "three." or q["answer"] == "Three." or q["answer"] == "three people" or q["answer"] == "Three people" or q["answer"] == "three people." or q["answer"] == "Three people.":
        data[i]["answer"] = "3"
    elif q["answer"] == "4." or q["answer"] == "four" or q["answer"] == "Four" or q["answer"] == "four." or q["answer"] == "Four." or q["answer"] == "four people" or q["answer"] == "Four people" or q["answer"] == "four people." or q["answer"] == "Four people.":
        data[i]["answer"] = "4"
    elif q["answer"] == "5." or q["answer"] == "five" or q["answer"] == "Five" or q["answer"] == "five." or q["answer"] == "Five." or q["answer"] == "five people" or q["answer"] == "Five people" or q["answer"] == "five people." or q["answer"] == "Five people.":
        data[i]["answer"] = "5"
    elif q["answer"] == "6." or q["answer"] == "six" or q["answer"] == "Six" or q["answer"] == "six." or q["answer"] == "Six." or q["answer"] == "six people" or q["answer"] == "Six people" or q["answer"] == "six people." or q["answer"] == "Six people.":
        data[i]["answer"] = "6"
    elif q["answer"] == "7." or q["answer"] == "seven" or q["answer"] == "Seven" or q["answer"] == "seven." or q["answer"] == "Seven." or q["answer"] == "seven people" or q["answer"] == "Seven people" or q["answer"] == "seven people." or q["answer"] == "Seven people.":
        data[i]["answer"] = "7"
    elif q["answer"] == "8." or q["answer"] == "eight" or q["answer"] == "Eight" or q["answer"] == "eight." or q["answer"] == "Eight." or q["answer"] == "eight people" or q["answer"] == "Eight people" or q["answer"] == "eight people." or q["answer"] == "Eight people.":
        data[i]["answer"] = "8"
    elif q["answer"] == "9." or q["answer"] == "nine" or q["answer"] == "Nine" or q["answer"] == "nine." or q["answer"] == "Nine." or q["answer"] == "nine people" or q["answer"] == "Nine people" or q["answer"] == "nine people." or q["answer"] == "Nine people.":
        data[i]["answer"] = "9"
    elif q["answer"] == "10." or q["answer"] == "ten" or q["answer"] == "Ten" or q["answer"] == "ten." or q["answer"] == "Ten." or q["answer"] == "ten people" or q["answer"] == "Ten people" or q["answer"] == "ten people." or q["answer"] == "Ten people.":
        data[i]["answer"] = "10"
    elif q["answer"] == "0." or q["answer"] == "None" or q["answer"] == "none" or q["answer"] == "None." or q["answer"] == "none." or q["answer"] == "0" or q["answer"] == "0 people" or q["answer"] == "None people" or q["answer"] == "none people" or q["answer"] == "None people." or q["answer"] == "none people." \
        or q["answer"] == "Zero" or q["answer"] == "zero" or q["answer"] == "Zero." or q["answer"] == "zero." or q["answer"] == "Zero people" or q["answer"] == "zero people" or q["answer"] == "Zero people." or q["answer"] == "zero people.":
        data[i]["answer"] = "0"

In [ ]:
samples = {}
for d in data:
    id = d["answer"]
    if id not in samples:
        samples[id] = []
    samples[id].append("v7w_"+str(d["image_id"])+".jpg")

In [ ]:
def generate_sample(count, image_ids):
    # Generate a sample for the given people count and image IDs
    if count == "1":
        text = "Find me an everyday image showing 1 person in the picture."
    else:
        text = f"Find me an everyday image with {count} people in the picture."

    sample = {
        "qry_text": f"{text}",
        "qry_img_path": "",
        "tgt_text": "<|image_1|> Represent the given image.",
        "tgt_img_path": f"{image_ids}"
    }
    return sample

In [ ]:
import random
# Generating negative samples
# The negative samples are generated based on the positive samples
final = []
for k, v in samples.items():
    nega = []
    nega_ids = []
    if k == "0":
        nega = ["3", "4", "5", "6", "7", "8", "9", "10", "over 10"]
    elif k == "1":
        nega = ["4", "5", "6", "7", "8", "9", "10", "over 10"]
    elif k == "2":
        nega = ["5", "6", "7", "8", "9", "10", "over 10"]
    elif k == "3":
        nega = ["0", "6", "7", "8", "9", "10", "over 10"]
    elif k == "4":
        nega = ["0", "1", "7", "8", "9", "10", "over 10"]
    elif k == "5":
        nega = ["0", "1", "2", "8", "9", "10", "over 10"]
    elif k == "6":
        nega = ["0", "1", "2", "3", "9", "10", "over 10"]
    elif k == "7":
        nega = ["0", "1", "2", "3", "4", "10", "over 10"]
    elif k == "8":
        nega = ["0", "1", "2", "3", "4", "5", "over 10"]
    elif k == "9":
        nega = ["0", "1", "2", "3", "4", "5", "6"]
    elif k == "10":
        nega = ["0", "1", "2", "3", "4", "5", "6", "7"]
    elif k == "over 10":
        nega = ["0", "1", "2", "3", "4", "5", "6", "7", "8"]
    for tag in nega:
        if tag in samples:
            nega_ids.extend(samples[tag])
    for i in v:
        nega_ids = random.sample(nega_ids, 99)
        ids = [i]
        ids.extend(nega_ids)
        sample = generate_sample(k, ids)
        final.append(sample)

In [ ]:
# Saving the final samples to a JSON file
os.makedirs("people_num", exist_ok=True)
output_file = os.path.join("people_num", "people_num_retrieval.json")
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(final, f, ensure_ascii=False, indent=4)